In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from PIL import Image
import json
import requests
from io import BytesIO
import numpy as np

Using TensorFlow backend.


In [4]:
from ann.image_operations import * 

# global variables
window_shape = (5, 5)
row_margin = window_shape[0] // 2
col_margin = window_shape[1] // 2

if __name__ == '__main__':
    x_data = []
    y_data = []
    with open('dataset/forest_photos_info.json', 'r') as fo:
        dataset = json.loads(fo.read())
        for i, image_key in enumerate(dataset):
            print(f"Processing Img: {image_key}")
            img = get_image_from_url(dataset[image_key]['url'])
            # img.show()
            # input()
            generate_data_set_from_image(img, x_data, y_data, window_shape)
            if i == 1:
                print(f"Generating data from {i+1} images")
                break

Processing Img: 27950322870_771d53a6e2.jpg
Processing Img: 27927622835_32b1427ea7.jpg
Generating data from 2 images


In [5]:
model = Sequential()

model.add(Dense(30, input_dim=window_shape[0] * window_shape[1], activation='sigmoid'))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(3))
          
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(np.array(x_data), np.array(y_data))

Epoch 1/1
324384/324384 [==============================] - 18s 55us/step - loss: 0.0289 - accuracy: 0.8063


### Now Let's try to predict the output image

In [96]:
list(dataset.keys())[-10:-1]

['1299455_fb3636f431.jpg',
 '7460265344_f3a60b2a6d.jpg',
 '5660119953_229aac8e2d.jpg',
 '14759591072_a6bfaa05ac.jpg',
 '26911681691_7b7494d1bc.jpg',
 '7793140654_4f8abfb57c.jpg',
 '13193044613_345bf475bd.jpg',
 '4692542180_71ebaa29a7.jpg',
 '4124025640_59d557c75a.jpg']

In [99]:
# test image
img_str = '4124025640_59d557c75a.jpg'
img = get_image_from_url(dataset[img_str]['url'])
np_img = np.asarray(img)
img.show()

# gray scale
gray_img = 0.21*np_img[:, :, 0] + 0.72*np_img[:, :, 1] + 0.07*np_img[:, :, 2]
gray_img = Image.fromarray(gray_img.astype(np.uint8))
gray_img.show()

In [100]:
x_test, y_test = [], []

generate_data_set_from_image(img, x_test, y_test, window_shape)
out = [model.predict(x_test, y_test)]
out = out * 255
out = out.reshape(np_img.shape - np.array([2*row_margin, 2*col_margin, 0]))
print(out.shape)

out_img = Image.fromarray(out.astype(np.uint8))
out_img.show()

(495, 329, 3)


#### end #####